In [40]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import datasets
import torchvision.transforms as transforms
import wandb
from torchvision.models import resnet50, ResNet50_Weights
import utils

In [41]:
labels = (utils.get_annotations('Data/data1/annotations'))
images = utils.get_images('Data/data1/images')

Loading annotations...
Loading images...


c:\Users\Christine Lagorce\Desktop\X M2\ComputerVision\utils.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


In [42]:
label_map = {'with_mask': 0, 'without_mask': 1}

#we keep only the RGB channels
new_label = []
new_image = []
for i,label in enumerate(labels):
    if label in label_map:
        new_label.append(label_map[label])
        a = images[i][:,:,0:3]
        new_image.append(images[i])

df = []
for i in range(len(new_label)):
    df.append((torch.tensor(images[i]), torch.tensor(new_label[i])))


df_train = df[:int(len(df)*0.8)]
df_test = df[int(len(df)*0.8):]

In [43]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download and load the training data
trainset = datasets.CIFAR10('./data', download=True, train=True, transform=transform)
trainset_raw = datasets.CIFAR10('./data', download=True, train=True)

testset = datasets.CIFAR10('./data', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 10000)

df_train = []
for i, data in enumerate(trainloader, 0):
    df_train.append(data)

for j, data in enumerate(testloader, 0):
    df_test.append(data)

100%|██████████| 170498071/170498071 [00:42<00:00, 3994937.54it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [44]:
def train_WB_GPU(model, optimizer, criterion, n_epochs=10, verbose=True, use_wandb=False):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    #preprocess = weights_resnet.transforms()
    # Loop over epochs
    for n_epochs in range(n_epochs):

        # Loop over data batches
        running_loss = 0
        for i, data in enumerate(df_train):
            if i % 100 == 0:
                print(i)
            # Training pass
            input, label = data
            input = input.to(device)
            #batch = preprocess(image).unsqueeze(0)
            label = label.to(device)
            #label.to(device)
            # --------------
            # Zero gradients
            optimizer.zero_grad() # Parce que dès fois, ex : RNN on veut garder l'info sur le dernier gradient calculé 
            # Forward pass
            prob = model(input)#batch as a argument
            # Compute loss
            loss = criterion(prob, label)
            # Backwards pass
            loss.backward()
            # Gradient step
            optimizer.step()


            running_loss += loss.item()

        loss_val = 0
        n_correct = 0
        n_tot_val = 0
        #Compute validation loss
        with torch.no_grad():
            model.eval()
            for xb_val, yb_val in df_test:
                xb_val = xb_val.to(device)
                #batch_val = preprocess(xb_val).unsqueeze(0)
                yb_val =  yb_val.to(device)
                n_val = xb_val.shape[0]
                n_tot_val += n_val
                prob_val = model(xb_val) #batch_val
                loss = criterion(prob_val, yb_val)
                loss_val += loss.item()

                pred_val = torch.argmax(prob_val, dim=1)
                is_correct = pred_val.eq(yb_val)
                n_correct += is_correct.sum().item()
            model.train()

        training_loss = round(running_loss/len(df_train),3)
        val_loss = round(loss_val/len(df_test),3)
        val_accuracy = n_correct/n_tot_val
        if verbose:
            print("Epoch number", n_epochs+1)
            print("------------")
            print("Training loss: ", training_loss)
            print("Val loss: ", val_loss)
            print("Val Accuracy: ", val_accuracy)
            wandb.log({"Training loss" :training_loss, "Val loss":  val_loss, "Val Accuracy" : val_accuracy, }) # POur donner ces infos à W&B ! 



In [46]:
#from torchvision.models import inception_v3, Inception_V3_Weights
from torchvision.models import resnet18, ResNet18_Weights

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
weights_resnet = ResNet18_Weights.IMAGENET1K_V1
model_resnet= resnet18(weights=weights_resnet)#.to(device)
model_resnet.train(True)
print("device :", device)
# for param in model_resnet.parameters():
#     param.requires_grad = False
model_resnet.fc = nn.Linear(2048, 10)#.to(device)
model_resnet.to(device)
#print(model_resnet)

criterion_inception = nn.CrossEntropyLoss() 
optimizer_inception= optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9) # We use a smaller learning rate for the fine tuning on our dataset

# We change the dataset to have the version of the dataset with the resize images (299,299)

train_WB_GPU(model_resnet , optimizer_inception, criterion_inception, n_epochs=3, verbose=True, use_wandb=False) 

device : cpu
0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x512 and 2048x10)